In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10,6)


In [3]:
df = pd.read_csv("Airbnb.csv")
print("Dataset shape:", df.shape)
print(df.head())


Dataset shape: (102599, 26)
        id                                              NAME      host id  \
0  1001254                Clean & quiet apt home by the park  80014485718   
1  1002102                             Skylit Midtown Castle  52335172823   
2  1002403               THE VILLAGE OF HARLEM....NEW YORK !  78829239556   
3  1002755                                               NaN  85098326012   
4  1003689  Entire Apt: Spacious Studio/Loft by central park  92037596077   

  host_identity_verified host name neighbourhood group neighbourhood  \
0            unconfirmed  Madaline            Brooklyn    Kensington   
1               verified     Jenna           Manhattan       Midtown   
2                    NaN     Elise           Manhattan        Harlem   
3            unconfirmed     Garry            Brooklyn  Clinton Hill   
4               verified    Lyndon           Manhattan   East Harlem   

        lat      long        country  ... service fee minimum nights  \
0  4

C:\Users\Ahad\AppData\Local\Temp\ipykernel_9120\36686419.py:1: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Airbnb.csv")


In [5]:
"""
Airbnb NYC — Complete Analysis Script
Author: Ahad Khan
Purpose: Clean, explore, and visualize the uploaded Airbnb dataset at Airbnb.csv
Outputs:
 - visuals/*.png  (plots)
 - output/cleaned_airbnb_nyc.csv
 - output/summary_stats.csv
Notes:
 - Works even if some expected columns are missing.
 - Designed to look like a real analyst project: modular, commented, and reproducible.
"""

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import matplotlib.ticker as mtick

# -------------------
# CONFIG
# ---------------------
DATA_PATH = Path("Airbnb.csv")   
OUTPUT_DIR = Path("./output")
VIS_DIR = Path("./visuals")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
VIS_DIR.mkdir(parents=True, exist_ok=True)

sns.set(style="whitegrid", font_scale=1.05)
plt.rcParams["figure.figsize"] = (10, 6)

# ---------------------------
# UTIL FUNCTIONS
# ---------------------------
def load_data(path: Path):
    """Load CSV with robust settings and return dataframe."""
    if not path.exists():
        raise FileNotFoundError(f"{path} not found. Please place dataset at this path.")
    # read with low_memory False to avoid dtype warnings
    df = pd.read_csv(path, low_memory=False)
    return df

def clean_column_names(df: pd.DataFrame):
    """Normalize column names: strip, lower, replace spaces with underscore."""
    df = df.copy()
    df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
    return df

def parse_currency_column(df: pd.DataFrame, colname: str):
    """Convert currency-like strings ($1,234 or 1234) to numeric. If column absent, skip."""
    if colname not in df.columns:
        return df
    # remove currency symbols, commas, parentheses
    df[colname] = (
        df[colname]
        .astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .str.replace(r"\(", "-", regex=True)
        .str.replace(r"\)", "", regex=True)
        .replace({"nan": np.nan})
    )
    # coerce to numeric
    df[colname] = pd.to_numeric(df[colname], errors="coerce")
    return df

def safe_fillna(df: pd.DataFrame, col, value):
    """Fillna if column exists."""
    if col in df.columns:
        df[col] = df[col].fillna(value)
    return df

def save_plot(fig, filename):
    """Save matplotlib figure to visuals directory."""
    path = VIS_DIR / filename
    fig.tight_layout()
    fig.savefig(path, dpi=160)
    plt.close(fig)
    print(f"Saved plot: {path}")

# ---------------------------
# LOAD & INITIAL INSPECTION
# ---------------------------
print("Loading dataset...")
df = load_data(DATA_PATH)
print("Original shape:", df.shape)
df = clean_column_names(df)
print("Columns:", df.columns.tolist())

# quick head
print("\nSample rows:")
display(df.head(3))

# ---------------------------
# BASIC CLEANING
# ---------------------------
# Drop exact duplicate rows
df = df.drop_duplicates()
print("After dropping duplicates:", df.shape)

# Normalize common columns
# Some common columns in this dataset: price, service_fee, minimum_nights, number_of_reviews, reviews_per_month, last_review, availability_365
df = parse_currency_column(df, "price")
df = parse_currency_column(df, "service_fee")   # only if present

# Convert certain numeric-like columns (if present) to numeric
numeric_cols = ["minimum_nights", "number_of_reviews", "reviews_per_month",
                "availability_365", "calculated_host_listings_count", "review_rate_number"]
for c in numeric_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

# last_review => datetime
if "last_review" in df.columns:
    df["last_review"] = pd.to_datetime(df["last_review"], errors="coerce")

# Fill frequent missing values sensibly
df = safe_fillna(df, "reviews_per_month", 0)
df = safe_fillna(df, "number_of_reviews", 0)
df = safe_fillna(df, "availability_365", 0)
df = safe_fillna(df, "calculated_host_listings_count", 0)

# Trim whitespace in string columns to avoid hidden duplicates
for c in df.select_dtypes(include=["object"]).columns:
    df[c] = df[c].astype(str).str.strip()

# Feature: availability ratio (0-1)
if "availability_365" in df.columns:
    df["availability_ratio"] = df["availability_365"].astype(float) / 365.0
else:
    df["availability_ratio"] = np.nan

# Filter unrealistic prices if price exists
if "price" in df.columns:
    # Many listings have extreme prices; for analysis clip to a reasonable range but keep full copy saved
    df = df[(df["price"].isna()) | ((df["price"] >= 5) & (df["price"] <= 1000))]

print("After cleaning shape:", df.shape)

# ---------------------------
# BASIC STATS & SAVE CLEANED CSV
# ---------------------------
clean_path = OUTPUT_DIR / "cleaned_airbnb_nyc.csv"
df.to_csv(clean_path, index=False)
print("Cleaned data saved to:", clean_path)

# Save quick summary stats (helpful for README)
summary = df.describe(include="all").T
summary_path = OUTPUT_DIR / "summary_stats.csv"
summary.to_csv(summary_path)
print("Summary stats saved to:", summary_path)

# ---------------------------
# EXPLORATORY PLOTS
# ---------------------------

# 1) Listings per borough (neighbourhood_group)
if "neighbourhood_group" in df.columns:
    fig, ax = plt.subplots(figsize=(9,5))
    order = df["neighbourhood_group"].value_counts().index
    sns.countplot(x="neighbourhood_group", data=df, order=order, ax=ax)
    ax.set_title("Listings per NYC Borough (neighbourhood_group)")
    ax.set_xlabel("Borough")
    ax.set_ylabel("Number of listings")
    save_plot(fig, "listings_per_borough.png")

# 2) Average price by borough
if "neighbourhood_group" in df.columns and "price" in df.columns:
    avg_price = df.groupby("neighbourhood_group")["price"].mean().sort_values()
    fig, ax = plt.subplots(figsize=(8,5))
    avg_price.plot(kind="bar", ax=ax)
    ax.set_title("Average Price by Borough")
    ax.set_ylabel("Average price (USD)")
    save_plot(fig, "avg_price_by_borough.png")

# 3) Price distribution (zoomed to < $500)
if "price" in df.columns:
    fig, ax = plt.subplots(figsize=(10,5))
    subset = df[df["price"] < 500]["price"].dropna()
    sns.histplot(subset, bins=60, kde=True, ax=ax)
    ax.set_title("Price distribution for listings (< $500)")
    ax.set_xlabel("Price (USD)")
    save_plot(fig, "price_distribution_under_500.png")

# 4) Price by room_type (boxplot)
if "room_type" in df.columns and "price" in df.columns:
    fig, ax = plt.subplots(figsize=(9,6))
    sns.boxplot(x="room_type", y="price", data=df[df["price"] < 1000], ax=ax)
    ax.set_ylim(0, 600)  # focus for visualization
    ax.set_title("Price by Room Type (zoomed)")
    save_plot(fig, "price_by_room_type_boxplot.png")

# 5) Top neighborhoods by listing count (horizontal bar)
if "neighbourhood" in df.columns:
    top_n = df["neighbourhood"].value_counts().head(15)
    fig, ax = plt.subplots(figsize=(10,7))
    sns.barplot(x=top_n.values, y=top_n.index, ax=ax, palette="viridis")
    ax.set_title("Top 15 Neighborhoods by Number of Listings")
    ax.set_xlabel("Number of listings")
    save_plot(fig, "top_15_neighborhoods.png")

# 6) Host listing distribution (how many hosts have how many properties)
if "calculated_host_listings_count" in df.columns:
    fig, ax = plt.subplots(figsize=(10,5))
    sns.histplot(df["calculated_host_listings_count"], bins=50, log_scale=(False, True), ax=ax)
    ax.set_xlim(0, df["calculated_host_listings_count"].quantile(0.99) * 1.1)
    ax.set_title("Distribution of number of listings per host (log y-axis)")
    ax.set_xlabel("Listings per host")
    save_plot(fig, "host_listings_distribution.png")

# 7) Correlation matrix for numeric features (select meaningful columns)
num_cols = [
    c for c in df.select_dtypes(include=[np.number]).columns
    if c not in ("id",)  # skip identifiers
]
if len(num_cols) >= 2:
    corr = df[num_cols].corr()
    fig, ax = plt.subplots(figsize=(11,9))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap="coolwarm", center=0, ax=ax)
    ax.set_title("Correlation matrix (numeric features)")
    save_plot(fig, "correlation_matrix.png")

# 8) Availability vs Price scatter (sample)
if "price" in df.columns and "availability_ratio" in df.columns:
    sample = df.sample(min(3000, len(df)), random_state=42)
    fig, ax = plt.subplots(figsize=(9,6))
    sns.scatterplot(x="availability_ratio", y="price", data=sample, alpha=0.5, ax=ax)
    ax.set_title("Availability ratio vs Price (sample)")
    ax.set_xlabel("Availability ratio (0-1)")
    ax.set_ylabel("Price (USD)")
    save_plot(fig, "availability_vs_price_scatter.png")

# 9) Reviews vs Price scatter (if reviews exist)
if "number_of_reviews" in df.columns and "price" in df.columns:
    sample = df.sample(min(3000, len(df)), random_state=24)
    fig, ax = plt.subplots(figsize=(9,6))
    sns.scatterplot(x="number_of_reviews", y="price", data=sample, alpha=0.5, ax=ax)
    ax.set_title("Number of reviews vs Price (sample)")
    ax.set_xlabel("Number of reviews")
    ax.set_ylabel("Price (USD)")
    save_plot(fig, "reviews_vs_price_scatter.png")

# 10) Simple map scatter (latitude/longitude colored by price) using matplotlib (no external map)
if "lat" in df.columns and "long" in df.columns and "price" in df.columns:
    sample_map = df.dropna(subset=["lat", "long", "price"]).sample(min(5000, len(df)), random_state=1)
    fig, ax = plt.subplots(figsize=(8,8))
    sc = ax.scatter(sample_map["long"], sample_map["lat"], c=sample_map["price"], s=8, cmap="inferno", alpha=0.6)
    ax.set_title("Geographic scatter of sample listings colored by price")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    cbar = plt.colorbar(sc, ax=ax)
    cbar.set_label("Price (USD)")
    save_plot(fig, "geo_scatter_price.png")

# ---------------------------
# DERIVE & SAVE KEY INSIGHTS (text)
# ---------------------------
insights = []

# Boroughs: price and counts
if "neighbourhood_group" in df.columns and "price" in df.columns:
    borough_counts = df["neighbourhood_group"].value_counts()
    borough_avg_price = df.groupby("neighbourhood_group")["price"].mean().sort_values(ascending=False)
    insights.append("Top boroughs by number of listings:\n" + borough_counts.head(5).to_string())
    insights.append("Average price by borough (high->low):\n" + borough_avg_price.round(2).to_string())

# Room type
if "room_type" in df.columns and "price" in df.columns:
    rt = df.groupby("room_type")["price"].agg(["count", "mean"]).sort_values("mean", ascending=False)
    insights.append("Room type summary (count, mean_price):\n" + rt.round(2).to_string())

# Hosts
if "calculated_host_listings_count" in df.columns:
    top_hosts = df.groupby("host_id")["calculated_host_listings_count"].max().sort_values(ascending=False).head(10)
    insights.append("Top hosts by listings (host_id : listings):\n" + top_hosts.to_string())

# Save insights to a simple text file
insights_path = OUTPUT_DIR / "key_insights.txt"
with open(insights_path, "w", encoding="utf-8") as f:
    f.write("\n\n".join(insights))
print("Insights saved to:", insights_path)

# ---------------------------
# OPTIONAL: Small price-prediction baseline (linear regression) - if user wants to show ML
# ---------------------------
do_ml = True
if do_ml and "price" in df.columns:
    # Keep it lightweight and transparent: simple model using a few numeric features
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import mean_absolute_error, r2_score

    # select features that exist
    candidate_features = [
        "minimum_nights", "number_of_reviews", "reviews_per_month",
        "calculated_host_listings_count", "availability_ratio"
    ]
    features = [c for c in candidate_features if c in df.columns]
    # Add encoded room_type and borough if present
    model_df = df.copy()
    if "room_type" in model_df.columns:
        model_df["room_type_enc"] = model_df["room_type"].astype("category").cat.codes
        features.append("room_type_enc")
    if "neighbourhood_group" in model_df.columns:
        model_df["borough_enc"] = model_df["neighbourhood_group"].astype("category").cat.codes
        features.append("borough_enc")

    # Drop rows with NaN in chosen features or price
    model_df = model_df.dropna(subset=features + ["price"])
    if len(model_df) > 100:
        X = model_df[features]
        y = model_df["price"]
        # sample to keep training quick
        X_sample, _, y_sample, _ = train_test_split(X, y, train_size=0.2, random_state=42)
        X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.25, random_state=42)
        lr = LinearRegression()
        lr.fit(X_train, y_train)
        y_pred = lr.predict(X_test)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)
        ml_summary = f"Baseline LinearRegression MAE: {mae:.2f}  R2: {r2:.3f}"
        print(ml_summary)
        # Save model metrics
        with open(OUTPUT_DIR / "ml_baseline_metrics.txt", "w") as f:
            f.write(ml_summary)
    else:
        print("Not enough rows for ML baseline.")

# ---------------------------
# FINISH
# ---------------------------
print("\nAll done ✅")
print(f"Plots in: {VIS_DIR.resolve()}")
print(f"Cleaned dataset in: {clean_path.resolve()}")
print(f"Insights text in: {insights_path.resolve()}")

# Short README snippet (append to output for convenience)
readme_snip = OUTPUT_DIR / "README_PROJECT_SNIPPET.txt"
readme_text = """Airbnb NYC Analysis — quick start
1. Cleaned CSV: cleaned_airbnb_nyc.csv
2. Visuals: see visuals/ directory
3. Key insights: key_insights.txt
4. To build a Tableau/Power BI dashboard, open cleaned_airbnb_nyc.csv and create map + borough/room-type dashboards.
"""
readme_snip.write_text(readme_text)
print("Project snippet saved to:", readme_snip)


Loading dataset...
Original shape: (102599, 26)
Columns: ['id', 'name', 'host_id', 'host_identity_verified', 'host_name', 'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country', 'country_code', 'instant_bookable', 'cancellation_policy', 'room_type', 'construction_year', 'price', 'service_fee', 'minimum_nights', 'number_of_reviews', 'last_review', 'reviews_per_month', 'review_rate_number', 'calculated_host_listings_count', 'availability_365', 'house_rules', 'license']

Sample rows:


,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,country,...,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN


After dropping duplicates: (102058, 26)
After cleaning shape: (84349, 27)
Cleaned data saved to: output\cleaned_airbnb_nyc.csv


C:\Users\Ahad\AppData\Local\Temp\ipykernel_9120\2106156607.py:148: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  summary = df.describe(include="all").T


Summary stats saved to: output\summary_stats.csv
Saved plot: visuals\listings_per_borough.png
Saved plot: visuals\avg_price_by_borough.png
Saved plot: visuals\price_distribution_under_500.png
Saved plot: visuals\price_by_room_type_boxplot.png
Saved plot: visuals\top_15_neighborhoods.png
Saved plot: visuals\host_listings_distribution.png
Saved plot: visuals\correlation_matrix.png
Saved plot: visuals\availability_vs_price_scatter.png
Saved plot: visuals\reviews_vs_price_scatter.png
Saved plot: visuals\geo_scatter_price.png
Insights saved to: output\key_insights.txt
Baseline LinearRegression MAE: 237.19  R2: -0.001

All done ✅
Plots in: C:\Users\Ahad\Untitled Folder 4\visuals
Cleaned dataset in: C:\Users\Ahad\Untitled Folder 4\output\cleaned_airbnb_nyc.csv
Insights text in: C:\Users\Ahad\Untitled Folder 4\output\key_insights.txt
Project snippet saved to: output\README_PROJECT_SNIPPET.txt
